In [1]:
!pip install redis
!pip install pymysql

  Using cached https://files.pythonhosted.org/packages/ac/a7/cff10cc5f1180834a3ed564d148fb4329c989cbb1f2e196fc9a10fa07072/redis-3.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ed/39/15045ae46f2a123019aa968dfcba0396c161c20f855f11dea6796bcaae95/PyMySQL-0.9.3-py2.py3-none-any.whl


In [2]:
import redis, json, time, pymysql
DB_Redis = redis.Redis(host='localhost', port=6379, db=0)
DB_MYSQL = pymysql.connect("localhost","tdr","tdr", "tdr")

!mysql -utdr -ptdr tdr < db.sql

def empty_mysql(table="films"):
    cur = DB_MYSQL.cursor()
    cur.execute('DELETE FROM {} WHERE 1=1'.format(table))
    DB_MYSQL.commit()
    cur.close()

def empty_redis(table = ""):
    if table.strip() == "":
        return DB_Redis.flushall()
    else:
        return DB_Redis.delete(*DB_Redis.keys("{}:*".format(table)))

cur = DB_MYSQL.cursor()
cur.execute("INSERT INTO films (title,year,genre_list,cast_list) VALUES('foo', 2019, 'bar', 'baz')")
DB_MYSQL.commit()
cur.close()
cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print(cur.fetchall())
cur.close()

mysql: [Warning] Using a password on the command line interface can be insecure.
(('foo', 2019, 'bar', 'baz'),)


In [3]:
with open('movies.json') as f:
    datas_temp = json.load(f)

#Suppression des doublons
datas_json = []
keys = []
for film in datas_temp:
    if film['title'] not in keys:
        keys.append(film['title'])
        datas_json.append(film)
print(datas_json[0])

{'title': 'After Dark in Central Park', 'year': 1900, 'cast': [], 'genres': []}


In [16]:
empty_redis()
t = time.time()
for film in datas_json:
    DB_Redis.set('films:{}'.format(film['title']), json.dumps(film))
t = time.time() - t
print('Insertions de {} films dans Redis : {:0.03f} secondes'.format(len(datas_json), t))

file = open("test.txt", "w")
for film in datas_json:
    file.write("SET \"films:{}\" \"{}\"\n". format(film['title'], json.dumps(film)))
t = time.time()
!cat test.txt | redis-cli --pipe
t = time.time() - t
print('Insertions bulk {} films dans Redis : {:0.03f} secondes'.format(len(DB_Redis.keys('*')), t))

Insertions de 26791 films dans Redis : 2.293 secondes
ERR Protocol error: unbalanced quotes in request
^C
Insertions bulk 26791 films dans Redis : 4.840 secondes


In [5]:
def mysql_insert_update(sql,values=None):
    try:
        # create a new cursor
        cur = DB_MYSQL.cursor()
        if values is None:
            cur.execute(sql)
        elif isinstance(values, list):      
            # execute the INSERT statement
            cur.executemany(sql,values)
        else:
            cur.execute(sql,value)
        # commit the changes to the database
        DB_MYSQL.commit()
        # close communication with the database
        cur.close()
    except (Exception) as error:
        print(error)

In [6]:
empty_mysql()
datas_tuple = []
for film in datas_json:
    datas_tuple.append((film['title'],str(film['year']),json.dumps(film['genres']),json.dumps(film['cast'])))
print(datas_tuple[0])
t = time.time()
mysql_insert_update("INSERT INTO films(title,year,genre_list,cast_list) VALUES(%s,%s,%s,%s)", datas_tuple)
t = time.time() - t
print('Insertions de {} films dans MySQL : {:0.03f} secondes'.format(len(datas_json), t))

('After Dark in Central Park', '1900', '[]', '[]')
Insertions de 26791 films dans MySQL : 0.446 secondes


In [7]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("SELECT title FROM films WHERE title LIKE 'X%'")
films = list(map(lambda title : title[0], cur.fetchall()))
t = time.time() - t
print('Recherche des films commençant par A ou a avec MySQL : {} films en {:0.03f} secondes'.format(cur.rowcount, t))
print(films)
cur.close()

Recherche des films commençant par A ou a avec MySQL : 17 films en 0.009 secondes
['X Marks the Spot', 'X the Unknown', 'X-15', 'X: The Man with the X-ray Eyes', 'Xanadu', 'X-Men', 'xXx', 'X2: X-Men United', 'xXx: State of the Union', 'X-Men: The Last Stand', 'X Games 3D: The Movie', 'X-Men Origins: Wolverine', 'X-Men: First Class', 'X-Men: Days of Future Past', 'X-Men: Apocalypse', 'xXx: Return of Xander Cage', 'XX']


In [8]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:[xX]*"))
films = map(lambda x : json.loads(x), films)
films = list(map(lambda x : x['title'], films))
t = time.time() - t
print('Recherche des films commençant par A ou a avec Redis : {} films en {:0.03f} secondes'.format(len(films), t))
print(films)

Recherche des films commençant par A ou a avec Redis : 17 films en 0.004 secondes
['X-Men: The Last Stand', 'xXx: State of the Union', 'xXx', 'X-Men: First Class', 'X Marks the Spot', 'X-Men', 'X-Men: Apocalypse', 'X Games 3D: The Movie', 'X: The Man with the X-ray Eyes', 'Xanadu', 'X-Men Origins: Wolverine', 'X-15', 'X-Men: Days of Future Past', 'X the Unknown', 'XX', 'X2: X-Men United', 'xXx: Return of Xander Cage']


In [9]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("SELECT title FROM films WHERE cast_list LIKE '%Sean Bean%'")
films = list(map(lambda title : title[0], cur.fetchall()))
t = time.time() - t
print('Recherche des films avec Sean bean avec MySQL : {} films en {:0.03f} secondes'.format(cur.rowcount, t))
print(films)
cur.close()

Recherche des films avec Sean bean avec MySQL : 15 films en 0.015 secondes
['Stormy Monday', 'Patriot Games', 'Ronin', "Don't Say a Word", 'The Lord of the Rings: The Fellowship of the Ring', 'Equilibrium', 'National Treasure', 'Flightplan', 'North Country', 'Silent Hill', 'Percy Jackson & the Olympians: The Lightning Thief', 'Mirror Mirror', 'Silent Hill: Revelation 3D', 'Jupiter Ascending', 'The Young Messiah']


In [10]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:*"))
films = map(lambda x : json.loads(x), films)
films = filter(lambda x : 'Sean Bean' in x['cast'], films)
films = list(map(lambda x : x['title'], films))
t = time.time() - t
print('Recherche des films avec Sean bean avec Redis : {} films en {:0.03f} secondes'.format(len(films), t))
print(films)

Recherche des films avec Sean bean avec Redis : 15 films en 0.225 secondes
['Patriot Games', 'Mirror Mirror', 'Silent Hill', 'Jupiter Ascending', 'Silent Hill: Revelation 3D', 'North Country', 'National Treasure', 'Ronin', 'The Young Messiah', 'Equilibrium', 'Percy Jackson & the Olympians: The Lightning Thief', "Don't Say a Word", 'The Lord of the Rings: The Fellowship of the Ring', 'Stormy Monday', 'Flightplan']


In [11]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("SELECT title FROM films WHERE year = 2001")
films = list(map(lambda title : title[0], cur.fetchall()))
t = time.time() - t
print('Recherche des films sortis en 2001 avec MySQL : {} films en {:0.03f} secondes'.format(cur.rowcount, t))
print(films[0:10])
cur.close()

Recherche des films sortis en 2001 avec MySQL : 212 films en 0.006 secondes
['15 Minutes', '3 A.M.', '3000 Miles to Graceland', 'A.I. Artificial Intelligence', 'The Affair of the Necklace', 'Ali', 'All Over the Guy', 'Along Came a Spider', 'The Amati Girls', "America's Sweethearts"]


In [12]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:*"))
films = map(lambda x : json.loads(x), films)
films = filter(lambda x : 2001 == x['year'], films)
films = list(map(lambda x : x['title'], films))
t = time.time() - t
print('Recherche des films sortis en 2001 avec Redis : {} films en {:0.03f} secondes'.format(len(films), t))
print(films[0:10])

Recherche des films sortis en 2001 avec Redis : 212 films en 0.219 secondes
['Hardball', 'Delivering Milo', 'Someone Like You', 'The Deep End', "God Didn't Give Me a Week's Notice", 'My Big Break', 'The Believer', 'Sugar & Spice', 'Crocodile Dundee in Los Angeles', 'Rat Race']


In [13]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:*"))
films = list(map(lambda x : json.loads(x), films))
for film in films:
    film['year'] = film['year'] + 1
    DB_Redis.set("films:{}".format(film['title']), json.dumps(film))
t = time.time() - t
print('Mise à jour des années de sorties de {} films avec Redis en {:0.03f} secondes'.format(len(films), t))
print(json.loads(DB_Redis.get('films:Avatar')))

Mise à jour des années de sorties de 26791 films avec Redis en 2.329 secondes
{'title': 'Avatar', 'year': 2010, 'cast': ['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Michelle Rodriguez', 'Stephen Lang', 'Laz Alonso', 'Giovanni Ribisi', 'Joel Moore', 'Wes Studi'], 'genres': ['Science Fiction']}


In [14]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("UPDATE films SET year = year + 1 WHERE 1=1")
cur.execute("SELECT * from films")
t = time.time() - t
print('Mise à jour des années de sorties de {} avec MySQL en {:0.03f} secondes'.format(cur.rowcount, t))
cur.execute("SELECT * FROM films WHERE title = 'Avatar'")
films = list(map(lambda film : {'title': film[1], 'year': film[2], 'genre_list': json.loads(film[3]), 'cast_list': json.loads(film[4])}, cur.fetchall()))
print(films[0])
cur.close()

Mise à jour des années de sorties de 26791 avec MySQL en 0.291 secondes
{'title': 'Avatar', 'year': 2010, 'genre_list': ['Science Fiction'], 'cast_list': ['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Michelle Rodriguez', 'Stephen Lang', 'Laz Alonso', 'Giovanni Ribisi', 'Joel Moore', 'Wes Studi']}


In [19]:
#Redis fournis sa propre commande de banchmark indiquant le nombre de requêtes éxécutées par secondes
#ici le nombre de requêtes éxécuté par seconde avec une seule connexions et une tailles de données de 1024 caractères
#pour les commandes GET et SET
!redis-benchmark -h localhost -p 6379 -c 1 -n 26000 -q -d 1024

PING_INLINE: 20108.28 requests per second
PING_BULK: 20186.34 requests per second
SET: 19388.52 requests per second
GET: 20233.46 requests per second
INCR: 19862.49 requests per second
LPUSH: 18895.35 requests per second
RPUSH: 18584.70 requests per second
LPOP: 18361.58 requests per second
RPOP: 18922.85 requests per second
SADD: 19862.49 requests per second
HSET: 18745.49 requests per second
SPOP: 19548.87 requests per second
LPUSH (needed to benchmark LRANGE): 18426.65 requests per second
LRANGE_100 (first 100 elements): 6550.77 requests per second
LRANGE_300 (first 300 elements): 1891.05 requests per second
LRANGE_500 (first 450 elements): 1161.03 requests per second
LRANGE_600 (first 600 elements): 842.21 requests per second
MSET (10 keys): 17356.47 requests per second



# Comparaisons de performances
Avec les quelques requêtes éxécutées ci-dessus, nous observons facilement une différence en terme de vitesse d'éxécution.

N'ayant pas de gestion de structure de données propre incorporé dans Redis, les données sont accessibles via le langage python (ce qui bride la vitesse d'éxécution et donc les performances de Redis par la technologie et le temps d'éxécution de Python)

On remarque que la seule fois où Redis est plus performant que MySQL est une recherche par regex dans les noms des films. On peut donc en déduire que l'on "concurrence" MySQL suivant comment sont agencées les clés valeurs. Autrement, ce genre de recherche serais potentiellement plus rapide si la clé portait l'information du titre du film (clé primaire) et du nom de la propriété recherchée (colonne).

# Script LUA

__Un interpréteur natif:__
Redis permet l'insertion et l'utilisation de scrypt Lua. En effet, un interpréteur Lua est nativement fournis dans Redis depuis la version 2.6.0.

Dans le cas d'un EVAL, le premier argument d'EVAL est un script Lua 5.1. Le script ne sert pas à définir une fonction Lua (et cela ne devrait pas être nécessaire). 

Cela permet notamment de mettre en place un programme Lua qui s’exécutera dans le contexte du serveur Redis.

Le deuxième argument d'EVAL permet de spécifier les arguments qui suivent le script et représentent les noms des clés Redis. 

Lua peut accéder aux arguments à l’aide de la variable globale KEYS sous la forme d’un tableau basé sur un (par exemple, KEYS [1], KEYS [2], etc).

# Transactions

Certaines commande redis permettent les transactions. Le fonctionnement est le suivant:
1. Démarrage d'un bloc de transaction avec la commande MULTI
1. Saisie des commandes de la transactions (SET, GET, KEYS, ...)
1. Éxécution des commandes saisies avec EXEC (équivalent du COMMIT en SQL)

Redis met également à disposition la commande DISCARD, qui permet d'annuler toutes les commandes d'une transactions. C'est l'équivalent du ROLLBACK en SQL.
Quelques autres particularités des transactions Redis:
- Les commandes EXEC et DISCARD terminent le bloc de transaction ouvert.
- La commande EXEC retourne les retours de chaques commandes de la transactions.
- Les transactions Redis sont atomiques, c'est à dire que soit toutes les commandes sont effectivement éxécutées, soit aucune ne l'est.

Example de transaction simple:
```shell=
> MULTI
OK
> INCR foo
QUEUED
> INCR bar
QUEUED
> EXEC
1) (integer) 1
2) (integer) 1
```
Example de transaction avec DISCARD:
```shell=
> SET foo 1
OK
> MULTI
OK
> INCR foo
QUEUED
> DISCARD
OK
> GET foo
"1"
```

# Partitionnement

Le partitionnement permet de fractionner les données en plusieurs instances Redis de telle sorte que chaque instance ne contienne qu'un seul sous-ensemble de clés.

Le partitionnement dans Redis a deux objectifs principaux:

* Il permet avant tout de créer des bases de données beaucoup plus volumineuses, en utilisant la mémoire combinée de plusieurs ordinateurs. Sans partitionnement, nous serions limité à la quantité de mémoire d'une seule machine.

-  Il permet de dimensionner la puissance de calcul sur plusieurs cœurs et plusieurs ordinateurs.  On utilise ainsi la bande passante réseau sur plusieurs ordinateurs et adaptateurs réseau comme un bus de communication.
       
Une alternative au partitionnement par plage est le partitionnement par hachage. Ce schéma fonctionne avec n'importe quelle clé, sans nécessiter de clé. 

Dans les cas les plus poussées et que nous ne développerons pas ici, nous pouvons également mentionner le hachage cohérent, forme avancée de partitionnement de hachage, qui nous semble être un sujet d'approfondissement pertinent.

Le partitionnement proposé par Redis possède néanmoins certaines limites qu'il est important de connaitre:

- Les opérations impliquant plusieurs clés ne sont généralement pas prises en charge. Par exemple, nous ne pouvons pas effectuer l'intersection entre deux ensembles s'ils sont stockés dans des clés mappées sur différentes instances Redis (il n'existe aucune manière directe de le faire).

- Les transactions Redis impliquant plusieurs clés ne peuvent pas être utilisées car la granularité du partitionnement est située dans ces clés, il est donc impossible de partager un jeu de données avec une seule clé énorme comme s'il s'agissait d'un seul très grand jeu trié.

- Lorsque le partitionnement est utilisé, la gestion des données est plus complexe. Par exemple, il nous faut gérer plusieurs fichiers RDB / AOF et pour effectuer une sauvegarde des données, nous devons agréger les fichiers de persistance de plusieurs instances et hôtes.

- Enfin, l'ajout et la suppression des capacités peut être complexe. Par exemple, le cluster Redis prend en charge principalement le rééquilibrage transparent des données avec la possibilité d'ajouter et de supprimer des nœuds au moment de l'exécution, mais d'autres systèmes, tel que le partitionnement côté client, ne premet pas cette fonctionnalité. Sur ce dernier point, une des solutions serait d'utiliser une technique de prédécoupage, chose que nous n'avons pas testé.


